In [1]:
import numpy as np
from copy import deepcopy

from fatiando import utils, gridder
from fatiando.mesher import TriaxialEllipsoid
from fatiando.gravmag import triaxial_ellipsoid
from numpy.testing import assert_almost_equal
from pytest import raises

C:\Users\Vanderlei\Documents\fatiando\fatiando\vis\mpl.py:70: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


In [2]:
# Local-geomagnetic field
F = 30000
inc = -3
dec = 57

gm = 1000  # geometrical factor
area = [-5.*gm, 5.*gm, -5.*gm, 5.*gm]
x, y, z = gridder.scatter(area, 300, z=0.)
axis_ref = gm  # reference semi-axis

# Triaxial ellipsoids used for testing
model = [TriaxialEllipsoid(x=-3*gm, y=-3*gm, z=3*axis_ref,
                           large_axis=axis_ref,
                           intermediate_axis=0.8*axis_ref,
                           small_axis=0.6*axis_ref,
                           strike=78, dip=92, rake=135,
                           props={'susceptibility tensor': [0.7, 0.7, 0.7,
                                                            90., 47., 13.]}),
         TriaxialEllipsoid(x=-gm, y=-gm, z=2.4*axis_ref,
                           large_axis=1.1*axis_ref,
                           intermediate_axis=0.7*axis_ref,
                           small_axis=0.3*axis_ref,
                           strike=4, dip=10, rake=5,
                           props={'susceptibility tensor': [0.2, 0.15, 0.05,
                                                            180, 19, -8.],
                                  'remanent magnetization': [3, -6, 35]}),
         TriaxialEllipsoid(x=3*gm, y=3*gm, z=4*axis_ref,
                           large_axis=1.5*axis_ref,
                           intermediate_axis=0.9*axis_ref,
                           small_axis=0.6*axis_ref,
                           strike=-58, dip=87, rake=49,
                           props={'remanent magnetization': [4.7, 39, 0]})]

In [3]:
def test_triaxial_ellipsoid_ignore_none():
    "Triaxial ellipsoid ignores model elements that are None"

    # forced physical property
    pmag = utils.ang2vec(7, -52, 13)

    # copy of the original model
    model_none = deepcopy(model)

    # force an element of the copy to be None
    model_none[1] = None

    # magnetic field produced by the original model
    # without the removed element
    bx = triaxial_ellipsoid.bx(x, y, z, [model[0], model[2]],
                               F, inc, dec, pmag=pmag)
    by = triaxial_ellipsoid.by(x, y, z, [model[0], model[2]],
                               F, inc, dec, pmag=pmag)
    bz = triaxial_ellipsoid.bz(x, y, z, [model[0], model[2]],
                               F, inc, dec, pmag=pmag)
    tf = triaxial_ellipsoid.tf(x, y, z, [model[0], model[2]],
                               F, inc, dec, pmag=pmag)

    # magnetic field produced by the copy
    bx2 = triaxial_ellipsoid.bx(x, y, z, model_none,
                                F, inc, dec, pmag=pmag)
    by2 = triaxial_ellipsoid.by(x, y, z, model_none,
                                F, inc, dec, pmag=pmag)
    bz2 = triaxial_ellipsoid.bz(x, y, z, model_none,
                                F, inc, dec, pmag=pmag)
    tf2 = triaxial_ellipsoid.tf(x, y, z, model_none,
                                F, inc, dec, pmag=pmag)

    assert_almost_equal(bx2, bx, decimal=15)
    assert_almost_equal(by2, by, decimal=15)
    assert_almost_equal(bz2, bz, decimal=15)
    assert_almost_equal(tf2, tf, decimal=15)


def test_triaxial_ellipsoid_ignore_missing_prop():
    "Triaxial ellipsoid ignores model without the needed properties"

    # forced physical property
    pmag = utils.ang2vec(2, -4, 17)

    # copy of the original model
    model_none = deepcopy(model)

    # remove the required properties of an element of the copy
    del model_none[1].props['susceptibility tensor']
    del model_none[1].props['remanent magnetization']

    # magnetic field produced by the original model
    # without an element
    bx = triaxial_ellipsoid.bx(x, y, z, [model[0], model[2]],
                               F, inc, dec, pmag=pmag)
    by = triaxial_ellipsoid.by(x, y, z, [model[0], model[2]],
                               F, inc, dec, pmag=pmag)
    bz = triaxial_ellipsoid.bz(x, y, z, [model[0], model[2]],
                               F, inc, dec, pmag=pmag)
    tf = triaxial_ellipsoid.tf(x, y, z, [model[0], model[2]],
                               F, inc, dec, pmag=pmag)

    # magnetic field produced by the copy
    bx2 = triaxial_ellipsoid.bx(x, y, z, model_none,
                                F, inc, dec, pmag=pmag)
    by2 = triaxial_ellipsoid.by(x, y, z, model_none,
                                F, inc, dec, pmag=pmag)
    bz2 = triaxial_ellipsoid.bz(x, y, z, model_none,
                                F, inc, dec, pmag=pmag)
    tf2 = triaxial_ellipsoid.tf(x, y, z, model_none,
                                F, inc, dec, pmag=pmag)

    assert_almost_equal(bx2, bx, decimal=15)
    assert_almost_equal(by2, by, decimal=15)
    assert_almost_equal(bz2, bz, decimal=15)
    assert_almost_equal(tf2, tf, decimal=15)

In [4]:
def test_triaxial_ellipsoid_self_demagnetization():
    "Self-demagnetization decreases the magnetization intensity"

    mag_with_demag = triaxial_ellipsoid.magnetization(model[1],
                                                      F, inc, dec,
                                                      demag=True)

    mag_without_demag = triaxial_ellipsoid.magnetization(model[1],
                                                         F, inc, dec,
                                                         demag=False)

    mag_with_demag_norm = np.linalg.norm(mag_with_demag, ord=2)
    mag_without_demag_norm = np.linalg.norm(mag_without_demag, ord=2)

    assert mag_with_demag_norm < mag_without_demag_norm

In [5]:
test_triaxial_ellipsoid_ignore_none()

In [6]:
test_triaxial_ellipsoid_ignore_missing_prop()

In [7]:
test_triaxial_ellipsoid_self_demagnetization()